<a href="https://colab.research.google.com/github/chrisporras/graddiss/blob/main/GradientDissentersHackathon2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hackathon 2023
ML for Biomedical Data Science
Team: Gradient Dissenters
Members: Audrey Lee, Christian Porras, Joy Jiang

## Install and load Kaggle mammography

In [ ]:
# install Kaggle public api
! pip install -q kaggle
from google.colab import files
files.upload()
# Choose the kaggle.json file that you downloaded
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
# Make directory named kaggle and copy kaggle.json file there.
!chmod 600 ~/.kaggle/kaggle.json
#Change the permissions of the file.
! kaggle datasets list

Saving kaggle.json to kaggle.json
mkdir: cannot create directory ‘/root/.kaggle’: File exists
ref                                                      title                                     size  lastUpdated          downloadCount  voteCount  usabilityRating  
-------------------------------------------------------  ---------------------------------------  -----  -------------------  -------------  ---------  ---------------  
themrityunjaypathak/covid-cases-and-deaths-worldwide     Covid Cases and Deaths WorldWide           8KB  2023-02-01 12:22:51          11361        376  1.0              
naveenkumar20bps1137/walmart-sales-analysis              WALMART SALES ANALYSIS                   119KB  2023-03-03 18:02:11           1794         49  0.9411765        
datascientistanna/customers-dataset                      Shop Customer Data                        23KB  2023-02-07 18:42:21           9785        222  1.0              
ulrikthygepedersen/co2-emissions-by-country             

In [ ]:
!kaggle competitions download -c mammography-image-patch-classification-2023

 99% 457M/460M [00:15<00:00, 18.9MB/s]
100% 460M/460M [00:15<00:00, 30.9MB/s]


In [ ]:
# data directory
!mkdir data
# unzip data there,
!unzip mammography-image-patch-classification-2023.zip -d data